## Esp32 Reciever Code
🔹 Steps

Make sure your ESP32 is connected to the same Wi-Fi network (your phone’s hotspot in your case).

Install the MicroPython firmware on ESP32 (you probably already have this).

Upload the umqtt.simple library (it comes with most MicroPython builds).

Write a script that:

Connects ESP32 to Wi-Fi

Connects to your phone’s MQTT broker (192.168.43.1)

Subscribes to emotion/detection

Prints the message ({"emotion":"Happy"}) to the terminal

In [ ]:
import network
import time
from umqtt.simple import MQTTClient
import ubinascii
import machine

# --- WiFi Config ---
WIFI_SSID = "HWA_HOTSPOT"
WIFI_PASS = "12345678"

# --- MQTT Config ---
BROKER_IP = "192.168.43.1"   # same as in your Python script
CLIENT_ID = ubinascii.hexlify(machine.unique_id())
TOPIC = b"emotion/detection"

# --- Connect WiFi ---
wifi = network.WLAN(network.STA_IF)
wifi.active(True)
wifi.connect(WIFI_SSID, WIFI_PASS)

print("Connecting to WiFi...", end="")
while not wifi.isconnected():
    print(".", end="")
    time.sleep(1)
print("\nWiFi Connected:", wifi.ifconfig())

# --- Callback when message is received ---
def sub_cb(topic, msg):
    print("Received:", msg.decode())  # print in Thonny/Putty

# --- Connect to MQTT Broker ---
client = MQTTClient(CLIENT_ID, BROKER_IP, 1883)
client.set_callback(sub_cb)
client.connect()
print("Connected to MQTT Broker")

# --- Subscribe ---
client.subscribe(TOPIC)
print("Subscribed to topic:", TOPIC.decode())

# --- Loop forever ---
try:
    while True:
        client.check_msg()  # check for new messages
        time.sleep(0.1)
except KeyboardInterrupt:
    client.disconnect()
    print("Disconnected")
